In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

c:\Users\jaesc2\GitHub\skforecast
0.19.0


In [2]:
from skforecast.preprocessing import reshape_series_exog_dict_to_long

In [3]:
series_dict = {
    "series_1": pd.Series(
        data=np.arange(3), index=pd.date_range(start="2020-01-01", periods=3, freq="D")
    ),
    "series_2": pd.Series(
        data=np.arange(3, 6),
        index=pd.date_range(start="2020-01-01", periods=3, freq="D"),
    ),
}

exog_dict = {
    "series_1": pd.DataFrame(
        data={
            "exog_1": np.arange(3),
            "exog_2": np.arange(10, 13),
            "exog_3": np.arange(20, 23),
        },
        index=pd.date_range(start="2020-01-01", periods=3, freq="D"),
    ),
    "series_2": pd.DataFrame(
        data={"exog_1": np.arange(3, 6), "exog_2": np.arange(13, 16)},
        index=pd.date_range(start="2020-01-01", periods=3, freq="D"),
    ),
}

series_df = reshape_series_exog_dict_to_long(series=series_dict, exog=exog_dict)
series_df

series_value  exog_1  exog_2  exog_3
series_id datetime                                        
series_1  2020-01-01             0       0      10    20.0
          2020-01-02             1       1      11    21.0
          2020-01-03             2       2      12    22.0
series_2  2020-01-01             3       3      13     NaN
          2020-01-02             4       4      14     NaN
          2020-01-03             5       5      15     NaN

In [14]:
series_dict = {
    "series_1": pd.Series(
        data=np.arange(3), index=pd.date_range(start="2020-01-01", periods=3, freq="D")
    ),
    "series_2": pd.Series(
        data=np.arange(3, 6),
        index=pd.date_range(start="2020-01-01", periods=3, freq="D"),
    ),
}

exog_dict = {
    "series_1": pd.DataFrame(
        data={
            "exog_1": np.arange(3),
            "exog_2": np.arange(10, 13),
            "exog_3": np.arange(20, 23),
        },
        index=pd.date_range(start="2020-01-01", periods=3, freq="D"),
    ),
    "series_2": pd.DataFrame(
        data={"exog_1": np.arange(3, 6), "exog_2": np.arange(13, 16)},
        index=pd.date_range(start="2020-01-01", periods=3, freq="D"),
    ),
}
result = reshape_series_exog_dict_to_long(
    series=series_dict,
    exog=exog_dict
)
expected_result = pd.DataFrame(
    data={
        "series_value": [0, 1, 2, 3, 4, 5],
        "exog_1": [0, 1, 2, 3, 4, 5],
        "exog_2": [10, 11, 12, 13, 14, 15],
        "exog_3": [20, 21, 22, np.nan, np.nan, np.nan],
    },
    index=pd.MultiIndex.from_product(
        [["series_1", "series_2"], pd.date_range("2020-01-01", periods=3)],
        names=["series_id", "datetime"],
    ),
)
expected_result

series_value  exog_1  exog_2  exog_3
series_id datetime                                        
series_1  2020-01-01             0       0      10    20.0
          2020-01-02             1       1      11    21.0
          2020-01-03             2       2      12    22.0
series_2  2020-01-01             3       3      13     NaN
          2020-01-02             4       4      14     NaN
          2020-01-03             5       5      15     NaN

In [6]:
pd.concat(series_dict, names=['series_id', 'datetime']).to_frame('series_value')

series_value
series_id datetime                
series_1  2020-01-01             0
          2020-01-02             1
          2020-01-03             2
series_2  2020-01-01             3
          2020-01-02             4
          2020-01-03             5

In [8]:
pd.concat(exog_dict, names=['series_id', 'datetime'])

exog_1  exog_2  exog_3
series_id datetime                          
series_1  2020-01-01       0      10    20.0
          2020-01-02       1      11    21.0
          2020-01-03       2      12    22.0
series_2  2020-01-01       3      13     NaN
          2020-01-02       4      14     NaN
          2020-01-03       5      15     NaN

In [10]:
exog = {
        'series_1': pd.Series([1, 2, 3], index=pd.date_range('2020-01-01', periods=3), name='exog_1'),
        'series_2': pd.Series([4, 5, 6], index=pd.date_range('2020-01-01', periods=3), name='exog_2')
    }
pd.concat(exog, names=['series_id', 'datetime']).to_frame()

0
series_id datetime     
series_1  2020-01-01  1
          2020-01-02  2
          2020-01-03  3
series_2  2020-01-01  4
          2020-01-02  5
          2020-01-03  6

In [13]:
exog = {
        'series_1': pd.Series([1, 2, 3], index=pd.date_range('2020-01-01', periods=3), name='exog_1').to_frame(),
        'series_2': pd.Series([4, 5, 6], index=pd.date_range('2020-01-01', periods=3), name='exog_2')
    }
exog = pd.concat(exog.values(), keys=exog.keys())
if isinstance(exog, pd.Series):
    exog = exog.to_frame('exog')
exog.index.names = ['series_id', 'datetime']
exog

exog_1  exog_2
series_id datetime                  
series_1  2020-01-01     1.0     NaN
          2020-01-02     2.0     NaN
          2020-01-03     3.0     NaN
series_2  2020-01-01     NaN     4.0
          2020-01-02     NaN     5.0
          2020-01-03     NaN     6.0

In [14]:
series = {
        'series_1': pd.Series([1, 2, 3], index=pd.date_range('2020-01-01', periods=3)),
        'series_2': pd.Series([4, 5, 6], index=pd.date_range('2020-01-01', periods=3))
    }
exog = {
    'series_1': pd.Series([1, 2, 3], index=pd.date_range('2020-01-01', periods=3), name='exog_1').to_frame(),
    'series_2': pd.Series([4, 5, 6], index=pd.date_range('2020-01-01', periods=3), name='exog_2')
}

reshape_series_exog_dict_to_long(
    series=series, exog=exog, series_col_name='exog_1'
)

ValueError: Column name conflict: 'exog_1' already exists in exog. Please choose a different `series_col_name` value.